In [1]:
# from sqlalchemy.orm import declarative_base
# Base = declarative_base()

In [2]:
!pip install --upgrade -q git+https://github.com/vnghia/defi-ai

In [3]:
!pip install catboost

In [4]:
import os

os.environ["HOST_URL"] = "http://51.91.251.0:3000"
os.environ["USER_ID"] = "f89fec0b-183b-4921-bf15-197101c14192"
os.environ["SQL_USERNAME"] = "postgres"
os.environ["SQL_PASSWORD"] = "^de<@TETh~}*;:/*"
os.environ["SQL_HOST"] = "34.155.175.170"
os.environ["SQL_PORT"] = "5432"
os.environ["SQL_SCRAPE_DATABASE"] = "scrape"

In [5]:
from catboost import CatBoostRegressor, Pool, cv, CatBoostClassifier

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from defi_ai import (
    Avatar,
    Hotel,
    Request,
    Response,
    City,
    Language,
    Sample,
    init_session,
)

Session = init_session()
session = Session()

In [7]:
df = Request.load_dataset(session, False)
sample = Sample.load_dataset(session)

cv_dataset = Pool(
    data=df.drop("price", axis=1),
    label=df[["price"]],
    cat_features=["language", "city", "group", "brand", "children_policy"],
)

In [10]:
cat_reg = CatBoostRegressor(iterations=1000, random_state=42, learning_rate=1)

In [11]:
cat_reg.fit(cv_dataset, metric_period=1000)
cat_reg.save_model("cat.cbm", pool=cv_dataset)

0:	learn: 27.9495243	total: 358ms	remaining: 5m 57s
999:	learn: 11.8305309	total: 3m 54s	remaining: 0us


In [12]:
!mkdir -p './sub'
submission = cat_reg.predict(sample)
sub_df = pd.DataFrame(submission, columns=["price"])
sub_df.index.name = "index"
sub_df.to_csv("./sub/submission.csv")